In [20]:
import torch 
import torchinfo

In [21]:
checkpoint = torch.load('/data1/jcampos/loss_landscape_taxonomy/workspace/checkpoint/different_knobs_subset_10/lr_0.1/lr_decay/ECON_10b/autoencoder/autoencoder_exp_0_epoch=00_loss=0.062.ckpt')

In [22]:
checkpoint.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])

In [23]:
checkpoint['state_dict'].keys()

odict_keys(['encoder.quant_input.x_min', 'encoder.quant_input.x_max', 'encoder.quant_input.act_scaling_factor', 'encoder.quant_input.pre_weight_scaling_factor', 'encoder.quant_input.identity_weight_scaling_factor', 'encoder.quant_relu.x_min', 'encoder.quant_relu.x_max', 'encoder.quant_relu.act_scaling_factor', 'encoder.quant_relu.pre_weight_scaling_factor', 'encoder.quant_relu.identity_weight_scaling_factor', 'encoder.conv.weight', 'encoder.conv.bias', 'encoder.conv.conv_scaling_factor', 'encoder.conv.weight_integer', 'encoder.conv.conv.weight', 'encoder.conv.conv.bias', 'encoder.enc_dense.weight', 'encoder.enc_dense.bias', 'encoder.enc_dense.fc_scaling_factor', 'encoder.enc_dense.weight_integer', 'encoder.enc_dense.bias_integer', 'decoder.dec_dense.weight', 'decoder.dec_dense.bias', 'decoder.convtrans2d1.weight', 'decoder.convtrans2d1.bias', 'decoder.convtrans2d2.weight', 'decoder.convtrans2d2.bias'])

In [24]:
import sys

In [25]:
# sys.path.append('workspace/src_econ/code/models/')
# sys.path.append('workspace/src_econ/code')

In [26]:
from models.q_autoencoder import AutoEncoder

In [27]:
model = AutoEncoder(
    accelerator='cpu',
    quantize=True,
    precision=[6, 7, 8,]
)

In [28]:
torchinfo.summary(model, input_size=(1, 1, 8, 8))  # (B, C, H, W)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [38]:
import tqdm 
from models.autoencoder_datamodule import AutoEncoderDataModule
from models.utils_pt import unnormalize, emd
from argparse import ArgumentParser
import numpy as np

In [44]:
def test_model(model, test_loader):
    """
    Our own testing loop instead of using the trainer.test() method so that we
    can multithread EMD computation on the CPU
    """
    model.eval()
    input_calQ_list = []
    output_calQ_list = []
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x.to(model.device)
            output = model(x)
            input_calQ = model.map_to_calq(x)
            output_calQ_fr = model.map_to_calq(output)
            input_calQ = torch.stack(
                [input_calQ[i] * model.val_sum[i] for i in range(len(input_calQ))]
            )  # shape = (batch_size, 48)
            output_calQ = unnormalize(
                torch.clone(output_calQ_fr), model.val_sum
            )  # ae_out
            input_calQ_list.append(input_calQ)
            output_calQ_list.append(output_calQ)
    input_calQ = np.concatenate([i_calQ.cpu() for i_calQ in input_calQ_list], axis=0)
    output_calQ = np.concatenate([o_calQ.cpu() for o_calQ in output_calQ_list], axis=0)
    start_time = time.time()
    with multiprocessing.Pool() as pool:
        emd_list = pool.starmap(emd, zip(input_calQ, output_calQ))
    print(f"EMD computation time: {time.time() - start_time} seconds")
    average_emd = np.mean(np.array(emd_list))
    print(f"Average EMD: {average_emd}")
    return average_emd

In [48]:
parser = ArgumentParser()
parser.add_argument("--process_data", action="store_true", default=False)
parser.add_argument("--max_epochs", type=int, default=100)
parser.add_argument("--save_dir", type=str, default="./pt_autoencoder_test")
parser.add_argument("--experiment_name", type=str, default="autoencoder")
parser.add_argument("--fast_dev_run", action="store_true", default=False)
parser.add_argument(
    "--accelerator", type=str, choices=["cpu", "gpu", "auto"], default="auto"
)
parser.add_argument("--checkpoint", type=str, default="", help="model checkpoint")
parser.add_argument("--train", action="store_true", default=False)
parser.add_argument("--evaluate", action="store_true", default=False)
parser.add_argument(
    "--quantize", 
    action="store_true", 
    default=False, 
    help="quantize model to 6-bit fixed point (1 signed bit, 1 integer bit, 4 fractional bits)"
)
parser = AutoEncoderDataModule.add_argparse_args(parser)
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--process_data] [--max_epochs MAX_EPOCHS]
                             [--save_dir SAVE_DIR]
                             [--experiment_name EXPERIMENT_NAME]
                             [--fast_dev_run] [--accelerator {cpu,gpu,auto}]
                             [--checkpoint CHECKPOINT] [--train] [--evaluate]
                             [--quantize] [--data_dir DATA_DIR]
                             [--data_file DATA_FILE]
                             [--num_workers NUM_WORKERS]
                             [--batch_size BATCH_SIZE]
ipykernel_launcher.py: error: argument --fast_dev_run: ignored explicit argument '/home/jcampos/.local/share/jupyter/runtime/kernel-v2-307572F5F3s9JttHm.json'


AssertionError: 

In [52]:
data_module = AutoEncoderDataModule()

# Need val_sum to compute EMD
_, val_sum = data_module.get_val_max_and_sum()
model.set_val_sum(val_sum)
data_module.setup("test")
test_results = test_model(model, data_module.test_dataloader())

TypeError: __init__() missing 1 required positional argument: 'data_file'